# Isolating target clusters and selecting best representatives

This notebook is an example of how to select representatives given a set of protein clusters of interest.

**Required:** Run the [**run pipeline**](./run_pipeline.ipynb) notebook first to make sure the required outputs are present. Also run **Step 1** from the [**hierarchical cluster plot**](./hierarchical_cluster_plot.ipynb) notebook as we will re-use some of those outputs.  
  

## Perform some exploration to select clusters

In [1]:
import pandas as pd
import pickle
import proteinclustertools.visuals.circle_plot as cp
import proteinclustertools.visuals.annotate as an
from matplotlib.colors import ListedColormap

Loading BokehJS ...

Read files from previous visualization.  
  
We will use multiple levels to filter the data as an example, but if you have a particular level of interest you can just pick representatives directly from them as well.

In [2]:
clusters=pd.read_csv("layouts/mmseqs_137-196-303.csv", dtype=str)
clusters.head()

id 137.0 196.0 303.0
0     0     0     0     7
1     1     0     0   187
2    10     0     0     3
3   100     0     0     3
4  1000     0     0   260

Load some annotations as picking criteria.

In [3]:
annot_table=pd.read_csv("../data/IPR001761_with_taxonomy.tsv", sep="\t")
annot_table.head()

Entry  Reviewed  Length  Organism (ID)  \
0  G3XD97  reviewed     340         208964   
1  P02924  reviewed     329          83333   
2  P0ACP1  reviewed     334          83333   
3  Q88HH7  reviewed     339         160488   
4  Q9KM69  reviewed     326         243277   

                                                 PDB Superkingdom  \
0                                                NaN     Bacteria   
1  1ABE;1ABF;1APB;1BAP;2WRZ;5ABP;6ABP;7ABP;8ABP;9...     Bacteria   
2                                    1UXC;1UXD;2IKS;     Bacteria   
3                                                NaN     Bacteria   
4                                                NaN     Bacteria   

           Phylum  
0  Pseudomonadota  
1  Pseudomonadota  
2  Pseudomonadota  
3  Pseudomonadota  
4  Pseudomonadota

Again, we will need to remap the numeric sequence ids back to their original.

In [4]:
header_map=pd.read_csv("../output/IPR001761_header_map.txt", dtype=str)
header_map['Entry']=header_map['header'].str.split("|").str[1]
header_map.head()

id                          header       Entry
0  0            sp|Q88HH7|PTXS_PSEPK      Q88HH7
1  1  tr|A0A0M1P0M0|A0A0M1P0M0_9BACL  A0A0M1P0M0
2  2  tr|A0A100YV14|A0A100YV14_TRASO  A0A100YV14
3  3  tr|A0A1C7IBZ4|A0A1C7IBZ4_9FIRM  A0A1C7IBZ4
4  4  tr|A0A1R4KFK1|A0A1R4KFK1_9LACT  A0A1R4KFK1

In [5]:
clusters=clusters.merge(header_map[['id','Entry']], on="id")
clusters.head()

id 137.0 196.0 303.0       Entry
0     0     0     0     7      Q88HH7
1     1     0     0   187  A0A0M1P0M0
2    10     0     0     3  A0A485EFL7
3   100     0     0     3  A0A1G9ETG5
4  1000     0     0   260  A0A943EYG6

Now the sequence level annotations can be matched directly to the clusters through this table.

Let's remake the plot to see the sequence space.

In [6]:
# load layout
layout=pickle.load(open("layouts/mmseqs_137-196-303.pkl", "rb"))

levels=clusters.columns[1:4]
# create annotations of taxonomy and review status
phylum=an.AnnotateClusters(clusters, levels, annot_table, "Phylum", "Entry", "Entry")
phylum_colors=an.ColorAnnot(phylum, cmap="Set3", top_n=10, saturation=.7, shuffle_colors_seed=1)

is_reviewed=annot_table['Reviewed']=='reviewed'
reviewed_only=clusters[clusters['Entry'].isin(annot_table[is_reviewed]['Entry'])]
reviewed=an.AnnotateClusters(reviewed_only, [levels[-1]], annot_table, "Reviewed", "Entry", "Entry")
reviewed_colors=an.ColorAnnot(reviewed, cmap=ListedColormap(['red']), saturation=.7)

In [8]:
plot, plot_data=cp.CirclePlot(layout, size=800,
                            annot_colors=phylum_colors,
                            outlines=reviewed_colors,
                            highlight_line_width=2
                            )

Pretend we are only interested in the main group of related sequences (big base cluster) and not in the outliers.  
  
We can filter for just the sequences in the big cluster (cluster 0 at cut-off of 137).  

In [32]:
non_outlier=clusters[clusters['137.0']=='0']
print(non_outlier.shape)
non_outlier.head()

(27730, 5)


id 137.0 196.0 303.0       Entry
0     0     0     0     7      Q88HH7
1     1     0     0   187  A0A0M1P0M0
2    10     0     0     3  A0A485EFL7
3   100     0     0     3  A0A1G9ETG5
4  1000     0     0   260  A0A943EYG6

Say we wanted to sample just non-reviewed clusters, up to 48 of the largest ones. For picking representatives, the big cluster (~28k sequences) is too large, so we should subdivide the cluster by moving up the cut-offs. We can select using the highest cut-off in the plot (303), that looks like it has at least 50 clusters. 

In [33]:
reviewed_clusters=non_outlier[non_outlier['Entry'].isin(annot_table[is_reviewed]['Entry'])]['303.0'].unique()
non_reviewed=non_outlier[~non_outlier['303.0'].isin(reviewed_clusters)]
print(non_reviewed.shape)
non_reviewed.head()

(21945, 5)


id 137.0 196.0 303.0       Entry
1      1     0     0   187  A0A0M1P0M0
4   1000     0     0   260  A0A943EYG6
5  10000     0     0    25  A0A246DUJ7
6  10001     0     0    42  A0A246DXA1
7  10002     0     0   159  A0A246E7G0

In [36]:
# rank by size
cluster_sizes=non_reviewed.groupby('303.0').size().sort_values(ascending=False)
print(cluster_sizes.head())
# select top 48
top_clusters=cluster_sizes.index[:48]

303.0
1    1986
2    1818
5     994
6     984
9     624
dtype: int64


## Pick cluster representatives

Given a list of cluster IDs, we can pick some representatives.

Again, because the sequences are tracked as numbers, prepare a dataframe to map the ids back that will be used by the representative selection code. It just needs to have the id (number) and header (desired output header) as columns.

In [47]:
remap_df=header_map.copy()
remap_df.drop(columns=['header'], inplace=True)
remap_df.rename(columns={'Entry':'header'}, inplace=True)
remap_df.head()

id      header
0  0      Q88HH7
1  1  A0A0M1P0M0
2  2  A0A100YV14
3  3  A0A1C7IBZ4
4  4  A0A1R4KFK1

#### HMM based method

In [ ]:
import proteinclustertools.tools.hmm_cluster_rep as hmm_rep

# this creates a zip file with all the results
fasta_file='../output/IPR001761_cleaned.fasta' # should be cleaned fasta with numerical headers
outfile='representatives/303.0_top48_hmm.zip'
hmm_rep.ParseClusterLevel(clusters, '303.0', fasta_file, top_clusters, outfile, id_map=header_map)

This creates a zip file containing 3 types of information:
1. 'cluster_top_hits.csv' tracks which clusters each selected representative is from.
2. 'cluster_top_hits.fasta' is just all cluster representatives in one fasta file.
3. For every cluster analyzed, a folder is created that contains the intermediate steps in the analysis.  
    a. The MSA of all sequences in the cluster  
    b. The HMM created from the MSA  
    c. The results of searching all sequences with the HMM. The user can traverse down this list for other hits (e.g., second or third best)  

#### Vector based method

In [69]:
import proteinclustertools.tools.vector_cluster_rep as vec_rep
import pickle

# need to load vector embeddings
embeddings=pickle.load(open("../output/IPR001761_embeddings.pkl", "rb"))
vreps=vec_rep.ClusterVectorRepresentative(clusters, '303.0', embeddings, top_clusters)

In [70]:
vreps.head()

cluster top_hit
0       1   27021
1      11    2922
2      12   12237
3      13   13450
4      14   16768

We can save the results, and convert the sequences to their original ids.

In [73]:
id_map=remap_df.set_index('id')['header'].to_dict()

Collect the sequences from the cleaned fasta file.

In [ ]:
from Bio import SeqIO

outpath='representatives/vector_picks/'

fasta='../output/IPR001761_cleaned.fasta' # should be cleaned fasta with numerical headers
seqs=[rec for rec in SeqIO.parse(fasta, "fasta") if rec.id in vreps['top_hit'].values]
# convert headers
for seq in seqs:
    seq.id=id_map[seq.id]
    seq.description=''

# write to file
SeqIO.write(seqs, outpath+'top_vector_picks.fasta', "fasta")

48

In [67]:
vreps['top_hit']=vreps['top_hit'].map(id_map)
print(vreps.shape)
vreps.head()

(48, 2)


cluster     top_hit
0       1  A0A7L4TJ50
1      11  A0A1B3XQA8
2      12  A0A1H0QTR6
3      13  A0A7I0K8F0
4      14  A0A6P2WNZ8

In [68]:
# save cluster-representative mapping
vreps.to_csv('representatives/vector_picks/vector_pick_clusters.csv', index=False)

# Sampling representatives across the entire sequence space  

The following demonstrates how to somewhat evenly sample representatives across sequence space.

The steps are:
1. Isolate desired broad sequence space.
2. Sample representatives from clusters (with max size limit).
3. If clusters are too large, go to the next cut-off, where they split into smaller clusters and try again.

All of this can be done using the cluster table.

In [2]:
import pandas as pd

# read cluster table of even cutoffs
clust_dir = '../output/mmseqs_clustering/'
cutoffs = ['100.0','150.0','200.0','250.0','300.0'] # can be higher resolution steps (e.g. every 10 bitscore), these are just examples

clust = None
for cutoff in cutoffs:
    clust_temp = pd.read_csv(f'{clust_dir}IPR001761_mmseqs_clusters_{cutoff}.csv', dtype=str).rename(columns={'cluster':cutoff})
    if clust is None:
        clust = clust_temp
    else:
        clust = clust.merge(clust_temp, on='id', how='outer')

clust.head()

id 100.0 150.0 200.0 250.0 300.0
0     0     0     0     0     0     7
1     1     0     0     0    84   187
2    10     0     0     0     0     3
3   100     0     0     0     0     3
4  1000     0     0     0   117   245

Pretend we want to ignore small outlier groups we see in the plot above (cutoff 137), we can filter to only include sequences in those large clusters (0,1,2).

In [5]:
# read the 137.0 clusters
clust_137 = pd.read_csv(f'{clust_dir}IPR001761_mmseqs_clusters_137.0.csv', dtype=str)
main_137_clusters = ['0','1','2']
clust_137 = clust_137[clust_137['cluster'].isin(main_137_clusters)]

# filter cluster table
clust = clust[clust['id'].isin(clust_137['id'])]

Next, we iteratively sample the table. We can adjust the min and max cluster size to tune the number of clusters we sample.

In [14]:
level_start='100.0'
index_start=cutoffs.index(level_start)

min_size=10
max_size=100
ids_picked=set()
cluster_picks_by_level={}
for i in range(index_start, len(cutoffs)):
    level=cutoffs[i]
    slice=clust[~clust['id'].isin(ids_picked)]
    if len(slice)==0:
        break
    sizes=slice.groupby(level).size()

    # get clusters with size between min_size and max_size
    cluster_picks_by_level[level]=sizes[(sizes>=min_size) & (sizes<=max_size)].index
    print('level', level, ':', len(cluster_picks_by_level[level]), 'clusters with size between', min_size, 'and', max_size)

    # get ids
    ids_picked.update(slice[slice[level].isin(cluster_picks_by_level[level])]['id'])
    # also add ids less than min_size, so they don't get picked up in the next level
    ids_picked.update(slice[~slice[level].isin(sizes[sizes>=min_size].index)]['id'])

level 100.0 : 0 clusters with size between 10 and 100
level 150.0 : 3 clusters with size between 10 and 100
level 200.0 : 23 clusters with size between 10 and 100
level 250.0 : 66 clusters with size between 10 and 100
level 300.0 : 106 clusters with size between 10 and 100


This gives us a dictionary of ids, which we can use to extract representatives.

In [17]:
cluster_picks_by_level

{'100.0': Index([], dtype='object', name='100.0'),
 '150.0': Index(['5', '6', '8'], dtype='object', name='150.0'),
 '200.0': Index(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '21', '22',
        '24', '25', '27', '28', '29', '30', '32', '35', '36', '8', '9'],
       dtype='object', name='200.0'),
 '250.0': Index(['100', '101', '102', '103', '107', '108', '109', '113', '23', '25',
        '26', '27', '29', '32', '33', '34', '35', '37', '38', '39', '40', '41',
        '43', '44', '45', '46', '47', '48', '51', '52', '53', '54', '56', '57',
        '58', '59', '60', '62', '63', '64', '65', '66', '67', '68', '71', '72',
        '74', '75', '76', '77', '78', '79', '80', '81', '82', '84', '86', '87',
        '90', '91', '93', '94', '95', '96', '97', '99'],
       dtype='object', name='250.0'),
 '300.0': Index(['100', '101', '103', '104', '109', '110', '112', '114', '115', '116',
        ...
        '82', '83', '84', '85', '87', '89', '90', '92', '94', '95'],
       dtype='obj

We can reuse the code above to extract the representatives. However, since there are now multiple cutoffs, each needs to be processed separately. Below are some scripts for convenience to avoid having to do each one by hand.

In [23]:
import proteinclustertools.tools.hmm_cluster_rep as hmm_rep
import proteinclustertools.tools.vector_cluster_rep as vec_rep
import importlib
importlib.reload(hmm_rep)
import pickle
import os

rep_levels={'levels':{},'vector_levels':{}}
vector_reps={}

# can do just one kind of representative, or both
do_HMM=True
do_vector=True
force_redo=True

for level, cluster_ids in cluster_picks_by_level.items():
    fasta_file='../output/IPR001761_cleaned.fasta' # should be cleaned fasta with numerical headers

    cluster_file=f"../output/mmseqs_clustering/IPR001761_mmseqs_clusters_{level}.csv" # path to the cluster file
    target_clusters=cluster_ids

    print(f'processing {len(target_clusters)} clusters at level {level}')

    out_path=f'representatives/hmm_multi_level_reps/{level}_reps.zip'
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    
    rep_levels['levels'][level]=out_path
    if do_HMM and (force_redo or not os.path.exists(out_path)):
        header_map=pd.read_csv('../output/IPR001761_header_map.txt', dtype=str)
        header_map['header']=header_map['header'].str.split('|').str[1] # optional, adjust headers when outputting picks
        hmm_rep.ParseClusterLevel(cluster_file, 'cluster', fasta_file, target_clusters, out_path, id_map=header_map)

    # get vectors
    if do_vector:
        vectors=pickle.load(open('../output/IPR001761_embeddings.pkl', 'rb'))
        clu=pd.read_csv(cluster_file, dtype={'id':str})
        rep=vec_rep.ClusterVectorRepresentative(clu, 'cluster', vectors, target_clusters)
        rep_levels['vector_levels'][level]=rep
        vector_reps[level]=rep

# save levels data
with open('representatives/rep_levels.pkl', 'wb') as f:
    pickle.dump(rep_levels, f)

# save vector representatives, currently just a dictionary of dataframes
if do_vector:
    with open('representatives/vector_full_space_reps.pkl', 'wb') as f:
        pickle.dump(vector_reps, f)

processing 0 clusters at level 100.0
(Warning reading fasta) Excluded 130 sequences with non-amino acid characters.
processing 3 clusters at level 150.0
(Warning reading fasta) Excluded 130 sequences with non-amino acid characters.
processing 23 clusters at level 200.0
(Warning reading fasta) Excluded 130 sequences with non-amino acid characters.
processing 66 clusters at level 250.0
(Warning reading fasta) Excluded 130 sequences with non-amino acid characters.
processing 106 clusters at level 300.0
(Warning reading fasta) Excluded 130 sequences with non-amino acid characters.


We can grab the representatives from the files.
  
First, the hmms representatives can be grabbed from the individual zip files.

In [25]:
# need rep levels, can load from here if previous step was already run
rep_levels=pickle.load(open('representatives/rep_levels.pkl', 'rb'))

In [ ]:
from zipfile import ZipFile
from Bio import SeqIO
import io

hmm_reps={}
with open('representatives/hmm_full_space_reps.fasta', 'w') as f:
    for level, path in rep_levels['levels'].items():
        print(level, path)
        # read zip file list files
        with ZipFile(path, 'r') as z:
            print(z.namelist())
            # read csv table
            with z.open('cluster_top_hits.fasta') as zf:
                with io.TextIOWrapper(zf, encoding="utf-8") as tf:
                    seqs=[seq for seq in SeqIO.parse(tf, 'fasta')]
                    # write to file
                    SeqIO.write(seqs, f, 'fasta')

    

100.0 representatives/hmm_multi_level_reps/100.0_reps.zip
['cluster_top_hits.csv', 'cluster_top_hits.fasta']
150.0 representatives/hmm_multi_level_reps/150.0_reps.zip
['5/cluster_5.aln', '5/cluster_5.hmm', '5/cluster_5.hits', '6/cluster_6.aln', '6/cluster_6.hmm', '6/cluster_6.hits', '8/cluster_8.aln', '8/cluster_8.hmm', '8/cluster_8.hits', 'cluster_top_hits.csv', 'cluster_top_hits.fasta']
200.0 representatives/hmm_multi_level_reps/200.0_reps.zip
['10/cluster_10.aln', '10/cluster_10.hmm', '10/cluster_10.hits', '11/cluster_11.aln', '11/cluster_11.hmm', '11/cluster_11.hits', '12/cluster_12.aln', '12/cluster_12.hmm', '12/cluster_12.hits', '13/cluster_13.aln', '13/cluster_13.hmm', '13/cluster_13.hits', '14/cluster_14.aln', '14/cluster_14.hmm', '14/cluster_14.hits', '15/cluster_15.aln', '15/cluster_15.hmm', '15/cluster_15.hits', '16/cluster_16.aln', '16/cluster_16.hmm', '16/cluster_16.hits', '17/cluster_17.aln', '17/cluster_17.hmm', '17/cluster_17.hits', '18/cluster_18.aln', '18/cluster_18.h

Now the vector representatives.

In [31]:
vec_rep = pickle.load(open('representatives/vector_full_space_reps.pkl', 'rb'))

vec_hits = []
for level, table in vec_rep.items():
    vec_hits.extend(table['top_hit'].values)

# read all sequences
seqs = [rec for rec in SeqIO.parse('../output/IPR001761_cleaned.fasta', 'fasta') if rec.id in vec_hits]
# convert headers
header_map=pd.read_csv('../output/IPR001761_header_map.txt', dtype=str)
header_map['header']=header_map['header'].str.split('|').str[1] # optional, adjust headers when outputting picks
id_map=header_map.set_index('id')['header'].to_dict()
for seq in seqs:
    seq.id=id_map[seq.id]
    seq.description=''

# write to file
with open('representatives/vector_full_space_reps.fasta', 'w') as f:
    SeqIO.write(seqs, f, 'fasta')